IMPORT LIBRARIES 📚

In [1]:
from sqlalchemy import create_engine
from sqlalchemy import URL
from sqlalchemy import text
from urllib.parse import quote_plus
from sqlalchemy.types import Integer, VARCHAR, CHAR
from sqlalchemy import ForeignKeyConstraint
import pandas as pd 

Connection and CREATE SCHEMA 📝

In [2]:
url_object = URL.create(
    "mysql+mysqlconnector",
    username="root",
    password="farzansql",
    host="localhost",
)

In [3]:
engine = create_engine(url_object)

In [5]:
with engine.connect() as conn : 
    conn.execute(text('CREATE Schema football;'))
    # conn.execute(text('DROP Schema football;'))

Read Clubs Data 📗

In [6]:
%pwd

'd:\\University\\Quera-BootCamp\\Quera-Football\\Phase2'

In [34]:
df_club = pd.read_csv('Phase1\data\club.csv', index_col=0)
df_club_players = pd.read_csv('Phase1\data\club_players.csv', index_col=0)
df_league_goals = pd.read_csv('Phase1\data\league_goals.csv', index_col=0)

Read Players Data 📗

In [49]:
df_players_data = pd.read_csv('Phase1\data\players_data.csv', index_col=0)
df_players_stats = pd.read_csv('Phase1\data\players_stats.csv', index_col=0)
df_players_transfers = pd.read_csv('Phase1\data\players_transfers.csv', index_col=0)

CREATE TABLE Country ⚽️

In [10]:
table_country = df_club.loc[:, ['CountryID', 'Country']].drop_duplicates()
table_country = table_country.rename(columns={'CountryID': 'country_id', 'Country': 'country_name'}).set_index('country_id')
table_country

,country_name
country_id,
189,England
40,Germany
75,Italy
50,France
157,Spain


In [11]:
with engine.connect() as conn:
    table_country.to_sql(name='country', con=conn, schema='football', if_exists='replace', chunksize=50, method='multi',
                          index_label= 'country_id', dtype={'country_id': Integer(), 'country_name': VARCHAR(15)})


CREATE TABLE League ⚽️

In [12]:
table_league = df_club.loc[:, ['League', 'CountryID']].drop_duplicates().assign(league_id=range(1, 6)).set_index('league_id')
table_league = table_league.rename(columns={'League': 'league_name', 'CountryID': 'country_id'})
table_league

,league_name,country_id
league_id,,
1,Premier League,189
2,Bundesliga,40
3,Serie A,75
4,Ligue 1,50
5,LaLiga,157


In [13]:
query_league = text("""
CREATE TABLE league (
league_id INT NOT NULL AUTO_INCREMENT,
league_name VARCHAR(30) NOT NULL,
country_id INT NOT NULL,
PRIMARY KEY (league_id),
FOREIGN KEY (country_id) REFERENCES country(country_id)
);""")

In [14]:
with engine.connect() as conn:
    conn.execute(text('USE football;'))
    conn.execute(query_league)

In [15]:
with engine.connect() as conn:
    table_league.to_sql(name='league', con=conn, schema='football', if_exists='append', chunksize=50, method='multi')

CREATE TABLE Club ⚽️

In [16]:
table_club = df_club.loc[:, ['ClubID', 'Club', 'Stadium', 'StadiumCap']].drop_duplicates()
table_club = table_club.rename(columns={'ClubID': 'club_id', 'Club': 'club_name', 'Stadium': 'stadium_name', 'StadiumCap': 'stadium_capacity'}).set_index('club_id')
table_club.head(3)

,club_name,stadium_name,stadium_capacity
club_id,,,
631,Chelsea FC,Stamford Bridge,40.853 Seats
281,Manchester City,Etihad Stadium,55.017 Seats
11,Arsenal FC,Emirates Stadium,60.704 Seats


In [17]:
with engine.connect() as conn:
    table_club.to_sql(name='club', con=conn, schema='football', if_exists='replace', chunksize=50, method='multi',
                          index_label= 'club_id', dtype={'club_id': Integer(), 'club_name': VARCHAR(30), 
                                                         'stadium_name': VARCHAR(30), 'stadium_capacity': VARCHAR(20)})


CREATE TABLE Season ⚽️

In [18]:
table_season = df_club.loc[:, ['Season']].drop_duplicates()
table_season = table_season.assign(season_id=range(1, len(table_season) + 1)).set_index('season_id').rename(columns={'Season': 'year'})
table_season

,year
season_id,
1,2015
2,2016
3,2017
4,2018
5,2019
6,2020
7,2021


In [19]:
with engine.connect() as conn:
    table_season.to_sql(name='season', con=conn, schema='football', if_exists='replace', chunksize=50, method='multi',
                          index_label= 'season_id', dtype={'season_id': Integer(), 'year': Integer()})


CREATE TABLE Season_info ⚽️

In [22]:
league_dict = dict(zip(table_league['league_name'], table_league.index))
league_dict

{'Premier League': 1, 'Bundesliga': 2, 'Serie A': 3, 'Ligue 1': 4, 'LaLiga': 5}

In [23]:
season_dict = dict(zip(table_season['year'], table_season.index))
season_dict

{2015: 1, 2016: 2, 2017: 3, 2018: 4, 2019: 5, 2020: 6, 2021: 7}

In [24]:
table_season_info = df_club.loc[:, ['Season', 'League', 'ClubID', 'Rank', 'Squad', 
                                    'avgAge', 'Foreigners', 'avgMarketValue', 'totalMarketValue', 
                                    'Club_income', 'Club_expenditure', 'Club_OverallBalance']]
                                    
table_season_info['League'] = table_season_info['League'].map(league_dict)
table_season_info['Season'] = table_season_info['Season'].map(season_dict)

table_season_info = table_season_info.rename(columns={'Season': 'season_id', 'League': 'league_id', 'ClubID': 'club_id',
                          'Rank': 'club_rank', 'Squad': 'squad', 'avgAge': 'avg_age',
                          'Foreigners': 'foreign_players', 'avgMarketValue': 'avg_marketValue',
                          'totalMarketValue': 'total_marketValue', 'Club_income': 'income',
                          'Club_expenditure': 'expenditure', 'Club_OverallBalance': 'overbalance'})

table_season_info = table_season_info.assign(season_info_id=range(1, len(table_season_info)+1)).set_index('season_info_id')
table_season_info.head(3)

,season_id,league_id,club_id,club_rank,squad,avg_age,foreign_players,avg_marketValue,total_marketValue,income,expenditure,overbalance
season_info_id,,,,,,,,,,,,
1,1,1,631,10.0,40,24.7,31,€13.84m,€553.50m,€87.49m,€96.50m,€-9.01m
2,1,1,281,4.0,38,25.4,27,€13.14m,€499.35m,€67.44m,€208.47m,€-141.03m
3,1,1,11,2.0,41,24.7,29,€11.17m,€457.85m,€2.50m,€26.50m,€-24.00m


In [ ]:
query_season_info = text("""
CREATE TABLE season_info (
season_info_id INT NOT NULL AUTO_INCREMENT,
season_id INT NOT NULL,
league_id INT NOT NULL,
club_id INT NOT NULL,
club_rank INT,
squad INT,
avg_age FLOAT,
foreign_players INT,
avg_marketValue VARCHAR(12),
total_marketValue VARCHAR(12),
income VARCHAR(12),
overbalance VARCHAR(12),
expenditure VARCHAR(12),
PRIMARY KEY (season_info_id),
FOREIGN KEY (season_id) REFERENCES season(season_id),
FOREIGN KEY (league_id) REFERENCES league(league_id),
FOREIGN KEY (club_id) REFERENCES club(club_id)
);""")

In [ ]:
with engine.connect() as conn:
    conn.execute(text('USE football;'))
    conn.execute(query_season_info)

In [25]:
with engine.connect() as conn:
    table_season_info.to_sql(name='season_info', con=conn, schema='football', if_exists='append', chunksize=50, method='multi')

CREATE TABLE Coach ⚽️

In [145]:
# mohammad

CREATE TABLE Coach_Season_info ⚽️

In [146]:
# mohammad

CREATE TABLE Victories ⚽️

In [147]:
# mohammad

CREATE TABLE Victories_Season_info ⚽️

In [148]:
# mohammad

CREATE TABLE Positions ⚽️

In [149]:
# Ali

CREATE TABLE Players ⚽️

In [26]:
table_players = df_players_data[["given_name", "full_name", "date_of_birth", "citizenship", "place_of_birth", "foot", "height"]]
table_players.head(3)

,given_name,full_name,date_of_birth,citizenship,place_of_birth,foot,height
player_id,,,,,,,
465955,Marcin Bulka,Marcin Bułka,"Oct 4, 1999",Poland,Płock,right,199.0
2998,Robert Huth,NaN,"Aug 18, 1984",Germany,Berlin,NaN,NaN
406633,Nya Kirby,Nya Jerome Kirby,"Jan 31, 2000",England,London,right,174.0


In [27]:
with engine.connect() as conn:
    table_players.to_sql(name="players", con=conn, schema="football", if_exists="replace", chunksize=50, method="multi",
                          index_label="player_id", dtype= {"player_id" : Integer(), "given_name" : VARCHAR(40), "full_name" : VARCHAR(70),
                                                           "date_of_birth" : VARCHAR(15), "citizenship" : VARCHAR(30), "place_of_birth" : VARCHAR(40),
                                                           "foot" : VARCHAR(6), "height" : Integer()})


CREATE TABLE Players_Season_info ⚽️

In [ ]:
# ali

CREATE TABLE Outfitters ⚽️

In [28]:
table_outfitters = df_players_data.loc[:, ['outfitter']].drop_duplicates().dropna()
table_outfitters = table_outfitters.assign(outfitter_id=range(1, len(table_outfitters) + 1)).set_index('outfitter_id').rename(columns={'outfitter': 'outfitter_name'})
table_outfitters

,outfitter_name
outfitter_id,
1,Nike
2,adidas
3,Puma
4,Under Armour
5,Umbro
6,New Balance
7,Uhlsport
8,Sells
9,Mizuno


In [29]:
with engine.connect() as conn:
    table_outfitters.to_sql(name="outfitter", con=conn, schema="football", if_exists="replace", chunksize=50, method="multi",
                              index_label = "outfitter_id", dtype = {"outfitter_id" : Integer(), "outfitter_name" : VARCHAR(15)})

CREATE TABLE Player_info ⚽️

In [ ]:
# zahra

CREATE TABLE Outfitter_Player_info ⚽️

In [ ]:
# Zhara

CREATE TABLE Stats ⚽️

In [ ]:
# Zahra

CREATE TABLE Transfer ⚽️

In [50]:
df_players_transfers.head(2)

,season,date,left,joined,mv,fee
player_id,,,,,,
465955,21/22,"Aug 5, 2021",Paris SG,OGC Nice,€800k,loan transfer
465955,20/21,"Jun 30, 2021",LB Châteauroux,Paris SG,€800k,End of loan


In [69]:
table_transfer = df_players_transfers.drop('date', axis=1).reset_index()
table_transfer = table_transfer.assign(transfer_id=range(1, len(table_transfer) + 1)).set_index('transfer_id')
table_transfer.rename(columns = {"season" : "transfer_date", 'left': 'club_left', 'joined': 'club_joined'}, inplace = True)
table_transfer.head(3)

,player_id,transfer_date,club_left,club_joined,mv,fee
transfer_id,,,,,,
1,465955,21/22,Paris SG,OGC Nice,€800k,loan transfer
2,465955,20/21,LB Châteauroux,Paris SG,€800k,End of loan
3,465955,20/21,Paris SG,LB Châteauroux,€900k,loan transfer


In [70]:
query_transfer = text("""
CREATE TABLE transfer (
transfer_id INT NOT NULL,
player_id INT NOT NULL,
transfer_date VARCHAR(15),
club_left VARCHAR(30),
club_joined VARCHAR(30),
mv VARCHAR(15),
fee VARCHAR(25),
PRIMARY KEY (transfer_id),
FOREIGN KEY (player_id) REFERENCES players(player_id)
);""")

In [67]:
with engine.connect() as conn:
    conn.execute(text('USE football;'))
    conn.execute(query_transfer)

In [71]:
with engine.connect() as conn:
    table_transfer.to_sql(name="transfer", con=conn, schema="football", if_exists="append", chunksize=50, method="multi")